<a href="https://colab.research.google.com/github/pdelfidali/SNDL/blob/main/HomeAssignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Home assignment 3 - Transfer Learning - Cats vs Dogs
## Piotr del Fidali | Kamil Zaremba

We decided to use transfer learning to classify cats and dogs. We will use the dataset from [Kaggle](https://www.kaggle.com/datasets/chetankv/dogs-cats-images), it contains 10000 cat images and 10000 dog images, divided into 80% training and 20 % test data sets. We will use Google Colab with GPU for faster training process.

In [ ]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))


Found GPU at: /device:GPU:0


In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"piotrdelfidali","key":"00f2a9bfc77e9e7b8b3a2341bc997be7"}'}

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d chetankv/dogs-cats-images
!unzip dogs-cats-images.zip

# Keras
In Keras there is ImageDataGenerator class that will be used. It rescales, resizes and labels the input images from given directory. We create one generator for each train and test data. We will use two base models:
- VGG16 which is a deep convolutional neural network architecture known for its simplicity, featuring 16 layers with convolutional layers stacked on top of each other in increasing depth.
- InceptionV3 which is a more complex architecture that utilizes inception modules, combining filters of different sizes in the same network layer to capture information at various scales.
On top of each base model, we add four layers. The First one is Flatten, we convert the multidimensional input layer and convert into a one-dimensional array. The Next three are Dense layers, the final one output size is 1 as we are doing binary classification.

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1. / 255)
validation_datagen = ImageDataGenerator(rescale=1. / 255)

In [ ]:
train_generator = train_datagen.flow_from_directory('/content/dataset/training_set',
                                                    batch_size = 32,
                                                    class_mode = 'binary',
                                                    target_size = (224, 224))


validation_generator =  validation_datagen.flow_from_directory( '/content/dataset/test_set',
                                                          batch_size  = 32,
                                                          class_mode  = 'binary',
                                                          target_size = (224, 224))

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [ ]:
from keras.models import Sequential
from keras.layers import Input, Dense, Flatten
from keras.applications.vgg16 import VGG16

keras_vgg16 = Sequential()
keras_vgg16.add(VGG16(weights="imagenet", include_top=False, input_shape=(224, 224, 3)))
keras_vgg16.add(Flatten())
keras_vgg16.add(Dense(128, activation="relu"))
keras_vgg16.add(Dense(64, activation="relu"))
keras_vgg16.add(Dense(1, activation="sigmoid"))

keras_vgg16.layers[0].trainable = False

keras_vgg16.summary()

58889256/58889256 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 128)               3211392   
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 17934401 (68.41 MB)
Trainable params: 3219713 (12.28 MB)
Non-trainable params: 14714688 (56.13 MB)
__________

In [ ]:
keras_vgg16.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
r = keras_vgg16.fit(
        train_generator,
        epochs=50,
        validation_data=validation_generator)

Epoch 1/50
250/250 [==============================] - 43s 168ms/step - loss: 0.0398 - accuracy: 0.9890 - val_loss: 0.6237 - val_accuracy: 0.8835
Epoch 2/50
250/250 [==============================] - 44s 174ms/step - loss: 0.0343 - accuracy: 0.9926 - val_loss: 0.9371 - val_accuracy: 0.8820
Epoch 3/50
250/250 [==============================] - 44s 176ms/step - loss: 0.0319 - accuracy: 0.9924 - val_loss: 0.3824 - val_accuracy: 0.9360
Epoch 4/50
250/250 [==============================] - 43s 170ms/step - loss: 0.0267 - accuracy: 0.9939 - val_loss: 0.4041 - val_accuracy: 0.9310
Epoch 5/50
250/250 [==============================] - 44s 177ms/step - loss: 0.0263 - accuracy: 0.9950 - val_loss: 0.4219 - val_accuracy: 0.9325
Epoch 6/50
250/250 [==============================] - 43s 171ms/step - loss: 0.0382 - accuracy: 0.9929 - val_loss: 0.4144 - val_accuracy: 0.9310
Epoch 7/50
250/250 [==============================] - 42s 168ms/step - loss: 0.0221 - accuracy: 0.9964 - val_loss: 0.5938 - val_ac

In [ ]:
keras_vgg16.evaluate(validation_generator)

63/63 [==============================] - 8s 131ms/step - loss: 0.8609 - accuracy: 0.9380


[0.8609157800674438, 0.9380000233650208]

For InceptionV3 we are doing the same model as for a VGG16 model, replacing only a base model.

In [ ]:
from keras.applications import InceptionV3

keras_inception = Sequential()
keras_inception.add(InceptionV3(weights="imagenet", include_top=False, input_shape=(224, 224, 3)))
keras_inception.add(Flatten())
keras_inception.add(Dense(128, activation="relu"))
keras_inception.add(Dense(64, activation="relu"))
keras_inception.add(Dense(1, activation="sigmoid"))

keras_inception.layers[0].trainable = False

keras_inception.summary()

In [ ]:
keras_inception.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
r = keras_inception.fit(
        train_generator,
        epochs=50,
        validation_data=validation_generator)

Epoch 1/50
250/250 [==============================] - 37s 115ms/step - loss: 0.8898 - accuracy: 0.9565 - val_loss: 0.0542 - val_accuracy: 0.9885
Epoch 2/50
250/250 [==============================] - 25s 99ms/step - loss: 0.1252 - accuracy: 0.9822 - val_loss: 0.1679 - val_accuracy: 0.9805
Epoch 3/50
250/250 [==============================] - 27s 107ms/step - loss: 0.0508 - accuracy: 0.9896 - val_loss: 0.1066 - val_accuracy: 0.9865
Epoch 4/50
250/250 [==============================] - 25s 99ms/step - loss: 0.0244 - accuracy: 0.9945 - val_loss: 0.0890 - val_accuracy: 0.9895
Epoch 5/50
250/250 [==============================] - 26s 106ms/step - loss: 0.0174 - accuracy: 0.9971 - val_loss: 0.1474 - val_accuracy: 0.9895
Epoch 6/50
250/250 [==============================] - 25s 101ms/step - loss: 0.0166 - accuracy: 0.9977 - val_loss: 0.0976 - val_accuracy: 0.9895
Epoch 7/50
250/250 [==============================] - 26s 102ms/step - loss: 6.5167e-05 - accuracy: 1.0000 - val_loss: 0.1441 - val_

In [ ]:
keras_inception.evaluate(validation_generator)

63/63 [==============================] - 5s 80ms/step - loss: 0.2125 - accuracy: 0.9890


[0.2124653160572052, 0.9890000224113464]

For both models results are very good, after only 50 epochs, model using VGG16 has an accuracy of 0.93800. The model using InceptionV3 did even better and after same number of epochs has an accuracy of 0.98900. The second model is not only better in terms of accuracy, but also it was trained a lot faster.

# PyTorch
For PyTorch we use only the Inception3 as a base model. We used the same network structure as in a Keras scenario. The implementation was a bit harder as we had to write more code, but it also allows to make a lot of tuning and custom logic for a specific task. Inception3 in PyTorch takes other input sizes (299x299) than in the Keras.

In [ ]:
import torch
import torch.nn as nn
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader

train_transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

valid_transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_dataset = datasets.ImageFolder('../data/dataset/training_set', transform=train_transform)
valid_dataset = datasets.ImageFolder('../data/dataset/test_set', transform=valid_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)

In [ ]:
class InceptionV3Model(nn.Module):
    def __init__(self):
        super(InceptionV3Model, self).__init__()
        self.base_model = models.inception_v3(pretrained=True)
        for param in self.base_model.parameters():
            param.requires_grad = False
        self.base_model.fc = nn.Sequential(
            nn.Linear(2048, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        if self.training:
            x, aux = self.base_model(x)
            return x
        else:
            x = self.base_model(x)
            return x

pytorch_inceptionv3 = InceptionV3Model()

D:\Studia\Semestr 2\SNDL\venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
D:\Studia\Semestr 2\SNDL\venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to C:\Users\piotr/.cache\torch\hub\checkpoints\inception_v3_google-0cc3c7bd.pth
100.0%


In [ ]:

# Loss and Optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.RMSprop(pytorch_inceptionv3.parameters(), lr=0.001)

# Training Loop
for epoch in range(10):
    pytorch_inceptionv3.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = pytorch_inceptionv3(inputs)
        loss = criterion(outputs, labels.float().unsqueeze(1))
        loss.backward()
        optimizer.step()

    pytorch_inceptionv3.eval()
    valid_loss = 0.0
    for inputs, labels in valid_loader:
        outputs = pytorch_inceptionv3(inputs)
        loss = criterion(outputs, labels.float().unsqueeze(1))
        valid_loss += loss.item()

    print(f"Epoch {epoch+1}, Validation Loss: {valid_loss / len(valid_loader)}")


Epoch 1, Validation Loss: 0.1084659102387608
Epoch 2, Validation Loss: 0.0755010639568643
Epoch 3, Validation Loss: 0.08579183453398329
Epoch 4, Validation Loss: 0.10519748406543855
Epoch 5, Validation Loss: 0.05948747944323316
Epoch 6, Validation Loss: 0.056588030926557994
Epoch 7, Validation Loss: 0.05583596926566864
Epoch 8, Validation Loss: 0.07319517400381821
Epoch 9, Validation Loss: 0.06331603878015091
Epoch 10, Validation Loss: 0.05585289882895138


In [ ]:
pytorch_inceptionv3.eval()
accuracy = 0.0
total = 0

for inputs, labels in valid_loader:
    outputs = pytorch_inceptionv3(inputs)
    predicted = outputs.round()
    total += labels.size(0)
    accuracy += (predicted == labels.unsqueeze(1)).sum().item()

print(f"Accuracy: {accuracy / total}")

Accuracy: 0.98


Similar to the implementation in Keras accuracy of the model using Inception3 accuracy is 98%. The training process took much more time.
To summarize, transfer learning can help to create models much faster than with the traditional approach. The Keras and PyTorch give almost the same value of accuracy metric. The pros of Keras are that is faster to develop and faster to train, while PyTorch gives more development options and modifications in network logic.
For this specific task, the InceptionV3 model did better in both time of execution and accuracy than the VGG16 model.